# Demo's for AZ-2005

This demo will showcase RAG patterns using Azure Cognitive Search

In [ ]:
#r "nuget:Microsoft.Extensions.Logging, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Debug, 8.0.0"

#r "nuget:Microsoft.SemanticKernel, 1.16.2"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core, 1.16.2-alpha"
#r "nuget:dotenv.net, 3.0.0"

#r "nuget:Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.16.2-alpha"
#r "nuget:Microsoft.SemanticKernel.Plugins.Memory, 1.16.2-alpha"

In [ ]:
using dotenv.net;
using dotenv.net.Utilities;

DotEnv.Load();

var openai_endpoint = EnvReader.GetStringValue("OPENAI_ENDPOINT");
var openai_key = EnvReader.GetStringValue("OPENAI_KEY");

var search_endpoint = EnvReader.GetStringValue("SEARCH_ENDPOINT");
var search_key = EnvReader.GetStringValue("SEARCH_KEY");

In [ ]:
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.Logging.Console;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Memory;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0020
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0050

var memory = new MemoryBuilder()
            .WithAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", openai_endpoint, openai_key)
            .WithMemoryStore(new AzureAISearchMemoryStore(search_endpoint, search_key))
            .Build();

var kernel = Kernel.CreateBuilder()
            .AddAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", openai_endpoint, openai_key)  
            .AddAzureOpenAIChatCompletion(
                "gpt-4o",         // Azure OpenAI Deployment Name. Make sure this is correct
                openai_endpoint,  // Azure OpenAI Endpoint
                openai_key)       // Azure OpenAI Key
            .Build();

kernel.ImportPluginFromObject(new Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin(memory));

## Use SK Memory to store some information into Azure Cognitive Search

In [ ]:
var memories = new Dictionary<string, string> 
{
    { "rec1", "My name is Wesley"},
    { "rec2", "I currently work as a Microsoft Technical Trainer"},
    { "rec3", "I've was born in Antwerp in 1978"},
    { "rec4", "I've visited the US a few times since 2007"},
    { "rec5", "I currently live in Kapellen"},
};

foreach (var item in memories)
{
    await memory.SaveInformationAsync(collection: "aboutMe", id: item.Key, text: item.Value);
}

memory.SaveReferenceAsync()

## Search memory using Vector Search (not using prompts)

In [ ]:
var query = "What is my name?";

var results = memory.SearchAsync(collection: "aboutMe", query: query, limit: 2);

await foreach (var result in results)
{
    Console.WriteLine(result.Metadata.Text);
    result.Display();
}

In [ ]:
var query = "What do I do for work?";

var results = memory.SearchAsync(collection: "aboutMe", query: query, limit: 2);

await foreach (var result in results)
{
    Console.WriteLine(result.Metadata.Text);
    result.Display();
}

## Grounding prompts with Memory "recall"

In [ ]:
const string prompt = @"
Considering these facts

About me: {{recall 'where did I grow up' collection='aboutMe'}}
About me: {{recall 'where do I live' collection='aboutMe'}}

Question: Do I live in the same town where I grep up?

Provide a concise answer ('Answer: ') and a separate explanation ('Explanation: '), in two lines.
";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");